<a href="https://colab.research.google.com/github/ParalelaUnsaac/G3-2020-1/blob/main/Paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!nvidia-smi


Mon Oct 19 14:17:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
#include "cuda_runtime.h"
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

#define MAXVALUE 10000

//------------------------------------

void numberGen(int N, int max, int* store)#Funcion Generar numeros
{
    int i;
    srand(time(0));#tiempo para generar numeros aleatorios
    for (i = 0; i < N; i++)
        store[i] = rand() % max; #Almacena en el arreglo
}
//------------------------------------

__global__ void countOdds(int* d, int N, int* odds)
{
    extern __shared__ int count[];

    int myID = blockIdx.x * blockDim.x + threadIdx.x;
    int localID = threadIdx.x;
    count[localID] = 0;
    if (myID < N)
        count[localID] = (d[myID] % 2);
    __syncthreads();

    // fase de reducción 
    int step = 1;
    while (((localID | step) < blockDim.x) && ((localID & step) == 0))
    {
        count[localID] += count[localID | step];
        step <<= 1;
        __syncthreads();
    }

   
    // adiciona al contador global
    if (localID == 0)
        atomicAdd(odds, count[0]);
}

//------------------------------------
int sharedSize(int b)#Generar
{
    return b * sizeof(int);
}

//------------------------------------

int main(int argc, char** argv)
{
    
    int N = 5000;
    int* ha, * hres, * da, * dres;   // punteros host (h*) y device (d*)

    ha = new int[N];
    hres = new int[1];

    cudaMalloc((void**)&da, sizeof(int) * N);
    cudaMalloc((void**)&dres, sizeof(int) * 1);

    numberGen(N, MAXVALUE, ha);

    cudaMemcpy(da, ha, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemset(dres, 0, sizeof(int));

    int blockSize, gridSize;
    cudaOccupancyMaxPotentialBlockSizeVariableSMem(&gridSize, &blockSize, (void*)countOdds, sharedSize, N);

    gridSize = ceil(1.0 * N / blockSize);
    printf("Grid : %i    Block : %i\n", gridSize, blockSize);
    countOdds << < gridSize, blockSize, blockSize * sizeof(int) >> > (da, N, dres);

    cudaMemcpy(hres, dres, sizeof(int), cudaMemcpyDeviceToHost);
    // chequeo
    int oc = 0;
    for (int i = 0; i < N; i++)
        if (ha[i] % 2){
            printf("%i ",ha[i]);
            oc++;
        }
    printf("\n");
            
    printf("Cantidad de numeros impares - Chequeo\n");
    printf("%i %i\n", *hres, oc);

    cudaFree((void*)da);
    cudaFree((void*)dres);
    delete[]ha;
    delete[]hres;
    cudaDeviceReset();
    getchar();
    return 0;
}

Grid : 5    Block : 1024
5643 7537 7019 7263 5299 1877 1225 6601 3865 3367 4367 5311 389 9255 5369 937 2255 2633 6161 4471 9395 9 1965 4707 6131 4947 7645 831 9911 3005 4551 8927 5319 4805 1961 6051 8323 4369 4403 7739 1149 7713 1253 267 5303 2865 1495 1961 4081 6731 5519 2389 7483 6741 3185 7629 7591 9719 2191 1787 2755 2055 2661 2211 5905 7815 187 949 7649 7981 1715 9837 1061 2819 4565 4153 9607 8953 5941 2397 6559 2677 6895 7845 1475 1001 7763 1107 5369 5465 9717 6875 2453 9815 27 1403 2293 2543 4397 2135 8931 6623 1527 3027 399 3467 343 551 9823 965 893 4793 6355 8649 3439 9595 1033 539 1445 9623 1223 7635 6439 6675 4385 8605 5279 2587 3523 215 1243 9687 5251 2481 5443 1599 1223 8099 6131 3601 9551 6629 3505 8071 5567 5847 9389 1227 9785 5643 8731 4755 3909 4697 5943 1279 3601 9879 2063 1799 9065 1837 1821 3369 53 2231 1139 8923 3825 8127 8703 981 6421 7337 4679 6301 3095 8457 6911 5467 1321 8621 4737 3097 2513 3661 2691 7667 8245 6777 2723 5579 8781 3009 3677 7135 319 2881 4903 26